# Install packages

In [23]:
# Install dependencies
# !pip3 uninstall numpy
!pip3 install --upgrade numpy==2.0.0
!pip3 install pandas
!pip3 install scikit-learn mlflow seaborn shap
!pip3 install bayesian-optimization
!pip3 install xgboost==2.1.2
!pip3 install optuna
!pip3 install optuna-integration[mlflow]



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: python310.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: python310.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: python310.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: python310.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: python310.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: python310.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: python310.exe -m pip install --upgrade pip


# Import package

In [ ]:
# Import library

import pandas as pd
import numpy as np

import mlflow
from mlflow import MlflowClient
from mlflow.models import infer_signature, make_metric
from optuna.integration.mlflow import MLflowCallback
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score, classification_report
from sklearn.model_selection import StratifiedKFold, GridSearchCV, RandomizedSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.inspection import permutation_importance
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier, plot_importance
import seaborn as sns
import matplotlib.pyplot as plt
import shap
import optuna
import os, json

# Prepare dataset
Because test dataset not have label, we must split train dataset to 2 parts. One for train and one for validate. We just do this on the first time

In [25]:
# Read dataset
# df = pd.read_csv('data/data.csv')
# train_test_data, validate_data = train_test_split(df, test_size=0.2, random_state=42)
# train_test_data.to_csv('data/train_.csv', index=False, header=True)
# validate_data.to_csv('data/validate.csv', index=False, header=True)

Now we check some information of dataset

In [26]:
# Check dataset
df = pd.read_csv('data/train_.csv')
df.head()
# print("Dataset column")
df.columns
# print("Summary of dataset info")
df.info()
# print("view dimensions of dataset")
df.shape

# for col in df.columns:
#   if df[col].dtype != 'object':  # Exclude non-numeric columns
#     min_val = df[col].min()
#     max_val = df[col].max()
#     print(f"Column: {col}")
#     print(f"Minimum: {min_val}")
#     print(f"Maximum: {max_val}")
#     print()

df.isnull().sum()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560110 entries, 0 to 1560109
Data columns (total 48 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   ID               1560110 non-null  int64  
 1   flow_duration    1560110 non-null  float64
 2   Header_Length    1404309 non-null  float64
 3   Protocol type    1404300 non-null  float64
 4   Duration         1404067 non-null  float64
 5   Rate             1403930 non-null  float64
 6   Srate            1404035 non-null  float64
 7   Drate            1404061 non-null  float64
 8   fin_flag_number  1560110 non-null  float64
 9   syn_flag_number  1560110 non-null  float64
 10  rst_flag_number  1404080 non-null  float64
 11  psh_flag_number  1404104 non-null  float64
 12  ack_flag_number  1560110 non-null  float64
 13  ece_flag_number  1404221 non-null  float64
 14  cwr_flag_number  1403991 non-null  float64
 15  ack_count        1404032 non-null  float64
 16  syn_count        1

ID                      0
flow_duration           0
Header_Length      155801
Protocol type      155810
Duration           156043
Rate               156180
Srate              156075
Drate              156049
fin_flag_number         0
syn_flag_number         0
rst_flag_number    156030
psh_flag_number    156006
ack_flag_number         0
ece_flag_number    155889
cwr_flag_number    156119
ack_count          156078
syn_count          156278
fin_count               0
urg_count               0
rst_count               0
HTTP               155993
HTTPS              156399
DNS                     0
Telnet             156044
SMTP               155832
SSH                156261
IRC                     0
TCP                156010
UDP                     0
DHCP                    0
ARP                156189
ICMP               155988
IPv                     0
LLC                     0
Tot sum            155800
Min                156138
Max                155961
AVG                     0
Std         

We see some cell have null value, we can not drop which rows have null cell because it to much. So we just fill all null value = -1

In [27]:
# Fill all null data as -1

val_dataset_ = pd.read_csv('data/validate.csv')
val_dataset = val_dataset_.fillna(-1, inplace=False)

print(df.isnull().sum())

data_n_null = df.fillna(-1, inplace=False)
data_n_null.isnull().sum()
data_n_null.head()
data_n_null.duplicated().sum()
data_n_null['Label'].unique().tolist()


X_val = val_dataset.drop(columns=['Label'])
y_val = val_dataset['Label']


ID                      0
flow_duration           0
Header_Length      155801
Protocol type      155810
Duration           156043
Rate               156180
Srate              156075
Drate              156049
fin_flag_number         0
syn_flag_number         0
rst_flag_number    156030
psh_flag_number    156006
ack_flag_number         0
ece_flag_number    155889
cwr_flag_number    156119
ack_count          156078
syn_count          156278
fin_count               0
urg_count               0
rst_count               0
HTTP               155993
HTTPS              156399
DNS                     0
Telnet             156044
SMTP               155832
SSH                156261
IRC                     0
TCP                156010
UDP                     0
DHCP                    0
ARP                156189
ICMP               155988
IPv                     0
LLC                     0
Tot sum            155800
Min                156138
Max                155961
AVG                     0
Std         

For tracking during training, we using MLflow. The software defined by container in mlflow folder

In [28]:
# Set mlflow as tracking server
ML_TRACKING_URL = "http://localhost:5000"
mlflow.set_tracking_uri(ML_TRACKING_URL)

# Train model
We train with some model with these steps
- We training with small part of dataset (0.2 or 0.3): dataset_frac
- We log artifacts, we see some column less contribute in  Feature Importance Score, so we delete it
- We train with full dataset, verify droped column is correct and need modify or not
- We use RandomizedSearchCV to search parameter
- We save best parameter to mlflow. With mlflow.sklearn.autolog, model and its metrics was save to model registry. We just download it and use

## Decision Tree


In [ ]:
tags = {
    "dataset_frac": 1.0,
    "test_size" : 0.2,
    "droped_column" : ['ID','IPv','DNS','IRC','DHCP','ARP','SMTP','cwr_flag_number','ece_flag_number','Telnet','Drate','psh_flag_number','rst_flag_number','LLC', 'TCP','SSH','HTTPS','ack_flag_number','Std','Tot size', 'ack_count'],
    "author": "Son Nguyen"
}

def dct_objective(trial):
    with mlflow.start_run(nested=True) as run:
        params = {
            "max_features": trial.suggest_int("max_features", 2, 50, step=2),
            "criterion": trial.suggest_categorical("criterion", ["gini", "entropy", "log_loss"]),
            "max_depth": trial.suggest_int("max_depth", 10, 1010, step=50),
            "min_samples_split": trial.suggest_int("min_samples_split", 2, 20, step=2),
            "min_samples_leaf": trial.suggest_int("min_samples_leaf", 2, 20, step = 2),
            "random_state" : 42
        }
       
        model = DecisionTreeClassifier(**params)

        mlflow.sklearn.autolog()
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='weighted')
        precision = precision_score(y_test, y_pred, average='weighted')
        recall = recall_score(y_test, y_pred, average='weighted')
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("f1_score", f1)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)


        feature_scores = pd.Series(model.feature_importances_, index=X_train.columns).sort_values(ascending=False)
        plt.figure(figsize=(20, 20))
        sns.barplot(x=feature_scores, y=feature_scores.index)
        plt.xlabel('Feature Importance Score')
        plt.ylabel('Features')
        plt.title("Visualizing Important Features")
        feature_importance_plot = "feature_importance.png"
        plt.savefig(feature_importance_plot, bbox_inches='tight')
        mlflow.log_artifact(feature_importance_plot)
        os.remove(feature_importance_plot)

        
        metrics_file = "model_summary.json"
        metrics = {
            "parameter" : {**params},
            "metrics" : {
                "f1" : f1,
                "precision" : precision,
                "accuracy" : accuracy,
                "recall" : recall
            }
            
        }
        
        with open(metrics_file, "w") as f:
            json.dump(metrics, f, indent=4)

        mlflow.log_artifact(metrics_file)
        os.remove(metrics_file)

        trial.set_user_attr("run_id", run.info.run_id)
    return f1


def dct_callback(study, frozen_trial):
    winner = study.user_attrs.get("winner", None)
    if study.best_value and winner != study.best_value:
        study.set_user_attr("winner", study.best_value)
        if winner:
            improvement_percent = (abs(winner - study.best_value) / study.best_value) * 100
            print(
                f"Trial {frozen_trial.number} achieved value: {frozen_trial.value} with "
                f"{improvement_percent: .4f}% improvement"
            )
        else:
            print(f"Initial trial {frozen_trial.number} achieved value: {frozen_trial.value}")


data = data_n_null.drop(columns=tags['droped_column'])
data_sample = data.sample(frac=tags['dataset_frac'])
X = data_sample.drop(columns=['Label'])
y = data_sample['Label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = tags['test_size'], random_state = 42)


mlflow.set_experiment("decision_tree")
optuna.logging.set_verbosity(optuna.logging.ERROR)



with mlflow.start_run(nested=True) as run:

    # mlflow.xgboost.autolog() can not put auto log here
    study = optuna.create_study(direction='maximize')
    study.optimize(dct_objective, n_trials=100, timeout=7200, callbacks=[dct_callback], show_progress_bar=True)
    # study.optimize(dct_objective, n_trials=100, timeout=100, callbacks=[dct_callback], show_progress_bar=True)
    
    best_trial = study.best_trial
    best_run_id = best_trial.user_attrs['run_id']
    # best_param = study.best_params
    best_value = study.best_value

    model_name = "Decision Tree"
    client = mlflow.tracking.MlflowClient()
    latest_ = client.get_latest_versions(model_name, stages=None)[0]

    model_uri = f"runs:/{best_run_id}/model"
    best_model = mlflow.register_model(model_uri, model_name)
    
    
    best_param = client.get_run(best_run_id).data.params
    if latest_:
        previous_f1_score = client.get_metric_history(latest_.run_id, "f1_score")[-1].value
        if previous_f1_score >= best_value:
            print(f"Last model is better. Current values {best_value}, latest values {previous_f1_score}")
        else:
    
            client.update_registered_model(
                name=model_name,
                description="Best moldel",
            )

            for key, value in best_param.items():
                client.set_model_version_tag(
                    name=model_name,
                    version=best_model.version,
                    key=key,
                    value=value
                )

            client.set_model_version_tag(
                name=model_name,
                version=best_model.version,
                key="values",
                value=best_value
            )

            for key, value in tags.items():
                mlflow.set_tag(key,value)
            mlflow.set_tag("job", "Decision Tree using optuna to search parameter")


  0%|          | 0/100 [00:00<?, ?it/s]

🏃 View run welcoming-swan-369 at: http://localhost:5000/#/experiments/5/runs/0f8e3ee641e04335b21339a6e36b19af
🧪 View experiment at: http://localhost:5000/#/experiments/5
Initial trial 0 achieved value: 0.8468287820706768
🏃 View run luminous-pig-19 at: http://localhost:5000/#/experiments/5/runs/4491782fa4974afdb5b43f57f1bb34cb
🧪 View experiment at: http://localhost:5000/#/experiments/5
Trial 1 achieved value: 0.919595941077566 with  7.9129% improvement
🏃 View run nimble-skink-793 at: http://localhost:5000/#/experiments/5/runs/59450eb87e4f4fb283a95ba9de8ee3f5
🧪 View experiment at: http://localhost:5000/#/experiments/5
Trial 2 achieved value: 0.9255817686520453 with  0.6467% improvement
🏃 View run crawling-sow-629 at: http://localhost:5000/#/experiments/5/runs/824698ad2473499b88eb47b0704c8c44
🧪 View experiment at: http://localhost:5000/#/experiments/5
Trial 3 achieved value: 0.926723055867559 with  0.1232% improvement
🏃 View run dapper-newt-514 at: http://localhost:5000/#/experiments/5/ru

d:\project\mlsec\final\project\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\project\mlsec\final\project\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


🏃 View run chill-bass-538 at: http://localhost:5000/#/experiments/5/runs/c69f2803924f4735b5db1fb19a8ed933
🧪 View experiment at: http://localhost:5000/#/experiments/5
🏃 View run traveling-smelt-874 at: http://localhost:5000/#/experiments/5/runs/c81af560dcfe46608617b5aaa24448db
🧪 View experiment at: http://localhost:5000/#/experiments/5
Trial 7 achieved value: 0.9293448766421688 with  0.0127% improvement


d:\project\mlsec\final\project\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\project\mlsec\final\project\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


...

🏃 View run marvelous-smelt-400 at: http://localhost:5000/#/experiments/5/runs/1676d75dfd5644ddaaae2929bbd316fe
🧪 View experiment at: http://localhost:5000/#/experiments/5
🏃 View run orderly-sponge-876 at: http://localhost:5000/#/experiments/5/runs/130d9c7bcaf8443595952a9ba4d25702
🧪 View experiment at: http://localhost:5000/#/experiments/5
🏃 View run exultant-ram-362 at: http://localhost:5000/#/experiments/5/runs/ae97e188a03e48998c525ad0ab6112e3
🧪 View experiment at: http://localhost:5000/#/experiments/5
🏃 View run legendary-stork-273 at: http://localhost:5000/#/experiments/5/runs/bb1a4e78be9b403681e6eb2d42d85899
🧪 View experiment at: http://localhost:5000/#/experiments/5
🏃 View run learned-cow-382 at: http://localhost:5000/#/experiments/5/runs/efdc5b3e2c004fcd934ec63fd327a26d
🧪 View experiment at: http://localhost:5000/#/experiments/5
🏃 View run carefree-cub-588 at: http://localhost:5000/#/experiments/5/runs/67575b77b01d4a8ab2a8f65fe96afb03
🧪 View experiment at: http://localhost:5000/#

C:\Users\SonNH\AppData\Local\Temp\ipykernel_7932\3928576652.py:108: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_ = client.get_latest_versions(model_name, stages=None)[0]
Registered model 'Decision Tree' already exists. Creating a new version of this model...
2024/12/05 22:13:20 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Decision Tree, version 5


🏃 View run learned-sloth-164 at: http://localhost:5000/#/experiments/5/runs/81279b835e9a4994b26b654153629e0b
🧪 View experiment at: http://localhost:5000/#/experiments/5


Created version '5' of model 'Decision Tree'.


🏃 View run calm-shad-475 at: http://localhost:5000/#/experiments/5/runs/6aa089706f7d4409ad7ce5f8c99867c0
🧪 View experiment at: http://localhost:5000/#/experiments/5


## Naive Bayes

In [30]:
tags = {
    "dataset_frac": 1.0,
    "random_state": 42,
    "test_size" : 0.2,
    "droped_column" : ['ID'],
    # "droped_column" : ['ID','IRC','DHCP','LLC','IPv','DNS','ece_flag_number','Drate','SMTP','Telnet','cwr_flag_number','ARP','SSH'],
    "author": "Son Nguyen"
}


def gnb_callback(study, frozen_trial):
    winner = study.user_attrs.get("winner", None)
    if study.best_value and winner != study.best_value:
        study.set_user_attr("winner", study.best_value)
        if winner:
            improvement_percent = (abs(winner - study.best_value) / study.best_value) * 100
            print(
                f"Trial {frozen_trial.number} achieved value: {frozen_trial.value} with "
                f"{improvement_percent: .4f}% improvement"
            )
        else:
            print(f"Initial trial {frozen_trial.number} achieved value: {frozen_trial.value}")

def gnb_objective(trial):
    with mlflow.start_run(nested=True) as run:
        params = {
            "var_smoothing": trial.suggest_float("max_samples",1e-9, 1e-1, log=True)
        }
        model = GaussianNB(**params)
        mlflow.sklearn.autolog()
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='weighted')
        precision = precision_score(y_test, y_pred, average='weighted')
        recall = recall_score(y_test, y_pred, average='weighted')
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("f1_score", f1)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)

        metrics_file = "model_summary.json"
        metrics = {
            "parameter" : {**params},
            "metrics" : {
                "f1" : f1,
                "precision" : precision,
                "accuracy" : accuracy,
                "recall" : recall
            }
            
        }
        
        with open(metrics_file, "w") as f:
            json.dump(metrics, f, indent=4)

        mlflow.log_artifact(metrics_file)
        os.remove(metrics_file)

        trial.set_user_attr("run_id", run.info.run_id)
    return f1

# Prepare data
data = data_n_null.drop(columns=tags['droped_column'])
data_sample = data.sample(frac=tags['dataset_frac'])
X = data_sample.drop(columns=['Label'])
y = data_sample['Label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = tags['test_size'], random_state = tags['random_state'])


mlflow.set_experiment("naive-bayes")
optuna.logging.set_verbosity(optuna.logging.ERROR)
with mlflow.start_run(nested=True) as run:

    # mlflow.xgboost.autolog() can not put auto log here
    study = optuna.create_study(direction='maximize')
    study.optimize(gnb_objective, n_trials=100, timeout=7200, callbacks=[gnb_callback], show_progress_bar=True)
    # study.optimize(gnb_objective, n_trials=100, timeout=100, callbacks=[gnb_callback], show_progress_bar=True)
    
    best_trial = study.best_trial
    best_run_id = best_trial.user_attrs['run_id']
    # best_param = study.best_params
    best_value = study.best_value

    model_name = "GaussianNB"
    client = mlflow.tracking.MlflowClient()
    latest_ = client.get_latest_versions(model_name, stages=None)[0]

    model_uri = f"runs:/{best_run_id}/model"
    best_model = mlflow.register_model(model_uri, model_name)
    
    
    best_param = client.get_run(best_run_id).data.params
    if latest_:
        previous_f1_score = client.get_metric_history(latest_.run_id, "f1_score")[-1].value
        if previous_f1_score >= best_value:
            print(f"Last model is better. Current values {best_value}, latest values {previous_f1_score}")
        else:
            client.update_registered_model(
                name=model_name,
                description="Best moldel",
            )

            for key, value in best_param.items():
                client.set_model_version_tag(
                    name=model_name,
                    version=best_model.version,
                    key=key,
                    value=value
                )

            client.set_model_version_tag(
                name=model_name,
                version=best_model.version,
                key="values",
                value=best_value
            )

            for key, value in tags.items():
                mlflow.set_tag(key,value)
            mlflow.set_tag("job", "GNB using optuna to search parameter")

  0%|          | 0/100 [00:00<?, ?it/s]

d:\project\mlsec\final\project\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\project\mlsec\final\project\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


🏃 View run dazzling-snake-951 at: http://localhost:5000/#/experiments/7/runs/e8e90936c2384390ab57b4a2233393be
🧪 View experiment at: http://localhost:5000/#/experiments/7
Initial trial 0 achieved value: 0.14948271287807147


d:\project\mlsec\final\project\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\project\mlsec\final\project\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


🏃 View run melodic-fish-86 at: http://localhost:5000/#/experiments/7/runs/e50639426cff4f4fb6addcfc80ea7484
🧪 View experiment at: http://localhost:5000/#/experiments/7


...

🏃 View run delightful-newt-68 at: http://localhost:5000/#/experiments/7/runs/b0b544085c024b80abc1f3b7a894eaf9
🧪 View experiment at: http://localhost:5000/#/experiments/7


d:\project\mlsec\final\project\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\project\mlsec\final\project\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\SonNH\AppData\Local\Temp\ipykernel_7932\316235869.py:87: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs

🏃 View run efficient-pug-612 at: http://localhost:5000/#/experiments/7/runs/378fb3d5ef7d491eb3a113bc2e6b4f7f
🧪 View experiment at: http://localhost:5000/#/experiments/7


Created version '2' of model 'GaussianNB'.


🏃 View run handsome-colt-774 at: http://localhost:5000/#/experiments/7/runs/095dad88cd0b4125b64005796ae5db35
🧪 View experiment at: http://localhost:5000/#/experiments/7


In [31]:
tags = {
    "dataset_frac": 1.0,
    "random_state": 42,
    "test_size" : 0.2,
    "droped_column" : ['ID','IRC','DHCP','LLC','IPv','DNS','ece_flag_number','Drate','SMTP','Telnet','cwr_flag_number','ARP','SSH'],
    "author": "Son Nguyen",
   
}


def rdf_objective(trial):
    with mlflow.start_run(nested=True) as run:
        params = {
            "n_estimators": trial.suggest_int("n_estimators", 50, 1050, step=100),
            "criterion": trial.suggest_categorical("criterion", ["gini", "entropy", "log_loss"]),
            "max_depth": trial.suggest_int("max_depth", 10, 1010, step=50),
            "min_samples_split": trial.suggest_int("min_samples_split", 2, 10),
            "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 20),
            "max_features": trial.suggest_int("max_features", 2, 30),
            "max_leaf_nodes": trial.suggest_int("max_leaf_nodes", 10, 100),
            "bootstrap": trial.suggest_categorical("bootstrap", [True]),
            "max_samples": trial.suggest_float("max_samples", 0.1, 1.0, step=0.1),
            "class_weight": trial.suggest_categorical("class_weight", ["balanced", "balanced_subsample", None]),
        }
       
        model = RandomForestClassifier(**params)
        mlflow.sklearn.autolog()
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='weighted')
        precision = precision_score(y_test, y_pred, average='weighted')
        recall = recall_score(y_test, y_pred, average='weighted')
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("f1_score", f1)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        
        feature_scores = pd.Series(model.feature_importances_, index=X_train.columns).sort_values(ascending=False)
        plt.figure(figsize=(20, 20))
        sns.barplot(x=feature_scores, y=feature_scores.index)
        plt.xlabel('Feature Importance Score')
        plt.ylabel('Features')
        plt.title("Visualizing Important Features")
        feature_importance_plot = "feature_importance.png"
        plt.savefig(feature_importance_plot, bbox_inches='tight')
        mlflow.log_artifact(feature_importance_plot)
        os.remove(feature_importance_plot)

        metrics_file = "model_summary.json"
        metrics = {
            "parameter" : {**params},
            "metrics" : {
                "f1" : f1,
                "precision" : precision,
                "accuracy" : accuracy,
                "recall" : recall
            }
            
        }

        with open(metrics_file, "w") as f:
            json.dump(metrics, f, indent=4)

        mlflow.log_artifact(metrics_file)
        os.remove(metrics_file)

        trial.set_user_attr("run_id", run.info.run_id)
    return f1

def rdf_callback(study, frozen_trial):
    winner = study.user_attrs.get("winner", None)
    if study.best_value and winner != study.best_value:
        study.set_user_attr("winner", study.best_value)
        if winner:
            improvement_percent = (abs(winner - study.best_value) / study.best_value) * 100
            print(
                f"Trial {frozen_trial.number} achieved value: {frozen_trial.value} with "
                f"{improvement_percent: .4f}% improvement"
            )
        else:
            print(f"Initial trial {frozen_trial.number} achieved value: {frozen_trial.value}")



data = data_n_null.drop(columns=tags['droped_column'])
data_sample = data.sample(frac=tags['dataset_frac'])
X = data_sample.drop(columns=['Label'])
y = data_sample['Label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = tags['test_size'], random_state = tags['random_state'])


mlflow.set_experiment("random-forest")
optuna.logging.set_verbosity(optuna.logging.ERROR)

with mlflow.start_run(nested=True) as run:

    # mlflow.xgboost.autolog() can not put auto log here
    study = optuna.create_study(direction='maximize')
    study.optimize(rdf_objective, n_trials=100, timeout=7200, callbacks=[rdf_callback], show_progress_bar=True)
    # study.optimize(rdf_objective, n_trials=100, timeout=100, callbacks=[rdf_callback], show_progress_bar=True)
    
    best_trial = study.best_trial
    best_run_id = best_trial.user_attrs['run_id']
    # best_param = study.best_params
    best_value = study.best_value

    model_name = "RandomForest-Classifier"
    client = mlflow.tracking.MlflowClient()
    latest_ = client.get_latest_versions(model_name, stages=None)[0]

    model_uri = f"runs:/{best_run_id}/model"
    best_model = mlflow.register_model(model_uri, model_name)
    
    
    best_param = client.get_run(best_run_id).data.params
    if latest_:
        previous_f1_score = client.get_metric_history(latest_.run_id, "f1_score")[-1].value
        if previous_f1_score >= best_value:
            print(f"Last model is better. Current values {best_value}, latest values {previous_f1_score}")
        else:
            client.update_registered_model(
                name=model_name,
                description="Best moldel",
            )

            for key, value in best_param.items():
                client.set_model_version_tag(
                    name=model_name,
                    version=best_model.version,
                    key=key,
                    value=value
                )

            client.set_model_version_tag(
                name=model_name,
                version=best_model.version,
                key="values",
                value=best_value
            )

            for key, value in tags.items():
                mlflow.set_tag(key,value)
            mlflow.set_tag("job", "random forest using optuna to search parameter")
    



  0%|          | 0/100 [00:00<?, ?it/s]

🏃 View run orderly-bass-352 at: http://localhost:5000/#/experiments/3/runs/655766bc2b724227818c2040dcd6d4ba
🧪 View experiment at: http://localhost:5000/#/experiments/3
Initial trial 0 achieved value: 0.8106853780563875
🏃 View run marvelous-skunk-86 at: http://localhost:5000/#/experiments/3/runs/032c84704a33437186c1e26c3b66f1c9
🧪 View experiment at: http://localhost:5000/#/experiments/3
Trial 1 achieved value: 0.8361783336803504 with  3.0487% improvement


C:\Users\SonNH\AppData\Local\Temp\ipykernel_7932\3809554955.py:110: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_ = client.get_latest_versions(model_name, stages=None)[0]
Registered model 'RandomForest-Classifier' already exists. Creating a new version of this model...
2024/12/06 03:02:26 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: RandomForest-Classifier, version 4


🏃 View run capricious-cod-131 at: http://localhost:5000/#/experiments/3/runs/21594e57fd8c4437a8605db3df0aafcd
🧪 View experiment at: http://localhost:5000/#/experiments/3
Trial 2 achieved value: 0.855878553417509 with  2.3018% improvement


Created version '4' of model 'RandomForest-Classifier'.


Last model is better. Current values 0.855878553417509, latest values 0.89875639483346
🏃 View run painted-bat-46 at: http://localhost:5000/#/experiments/3/runs/ea787c07655c4f918d6f2909aaeeac68
🧪 View experiment at: http://localhost:5000/#/experiments/3


## XGBoost

In [32]:
tags = {
    "dataset_frac": 1.0,
    "random_state": 42,
    "test_size" : 0.2,
    "droped_column" : ['ID','IPv','Drate','Telnet','SMTP','ARP','cwr_flag_number','ece_flag_number','fin_flag_number','SSH','psh_flag_number','rst_flag_number'],
    "author": "Son Nguyen"
}

def xgboost_objective(trial):
    with mlflow.start_run(nested=True) as run:
        params = {
            "tree_method" : "hist",
            "device" : "cuda",
            "objective": "reg:squarederror",
            "n_estimators": 1000,
            "verbosity": 0,
            "eval_metric" : ["rmse", "mae", "mape", "logloss","error","auc"],
            "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
            "max_depth": trial.suggest_int("max_depth", 1, 10),
            "subsample": trial.suggest_float("subsample", 0.05, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.05, 1.0),
            "min_child_weight": trial.suggest_int("min_child_weight", 1, 20),
        }
        model = XGBClassifier(**params)
        mlflow.xgboost.autolog()
        model.fit(X_train, y_train, verbose=False)
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='weighted')
        precision = precision_score(y_test, y_pred, average='weighted')
        recall = recall_score(y_test, y_pred, average='weighted')
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("f1_score", f1)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)

        metrics_file = "model_summary.json"
        metrics = {
            "parameter" : {**params},
            "metrics" : {
                "f1" : f1,
                "precision" : precision,
                "accuracy" : accuracy,
                "recall" : recall
            }
            
        }
        
        with open(metrics_file, "w") as f:
            json.dump(metrics, f, indent=4)

        mlflow.log_artifact(metrics_file)
        os.remove(metrics_file)

        trial.set_user_attr("run_id", run.info.run_id)
    return f1

def champion_callback(study, frozen_trial):
    winner = study.user_attrs.get("winner", None)
    if study.best_value and winner != study.best_value:
        study.set_user_attr("winner", study.best_value)
        if winner:
            improvement_percent = (abs(winner - study.best_value) / study.best_value) * 100
            print(
                f"Trial {frozen_trial.number} achieved value: {frozen_trial.value} with "
                f"{improvement_percent: .4f}% improvement"
            )
        else:
            print(f"Initial trial {frozen_trial.number} achieved value: {frozen_trial.value}")

data = data_n_null.drop(columns=tags['droped_column'])
data_sample = data.sample(frac=tags['dataset_frac'])
X = data_sample.drop(columns=['Label'])
y = data_sample['Label']
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = tags['test_size'], random_state = tags['random_state'])

mlflow.set_experiment("xgboost")
optuna.logging.set_verbosity(optuna.logging.ERROR)
with mlflow.start_run(nested=True) as run:

    # mlflow.xgboost.autolog() can not put auto log here
    study = optuna.create_study(direction='maximize')
    study.optimize(xgboost_objective, n_trials=100, timeout=7200, callbacks=[champion_callback], show_progress_bar=True)
    
    best_trial = study.best_trial
    best_run_id = best_trial.user_attrs['run_id']
    # best_param = study.best_params
    best_value = study.best_value

    model_name = "XGBoost-Classifier"
    client = mlflow.tracking.MlflowClient()
    latest_ = client.get_latest_versions(model_name, stages=None)[0]

    model_uri = f"runs:/{best_run_id}/model"
    best_model = mlflow.register_model(model_uri, model_name)
    
    best_param = client.get_run(best_run_id).data.params
    if latest_:
        previous_f1_score = client.get_metric_history(latest_.run_id, "f1_score")[-1].value
        if previous_f1_score >= best_value:
            print(f"Last model is better. Current values {best_value}, latest values {previous_f1_score}")
        else:
            client.update_registered_model(
                name=model_name,
                description="Best moldel",
            )

            for key, value in best_param.items():
                client.set_model_version_tag(
                    name=model_name,
                    version=best_model.version,
                    key=key,
                    value=value
                )

            client.set_model_version_tag(
                name=model_name,
                version=best_model.version,
                key="values",
                value=best_value
            )

            for key, value in tags.items():
                mlflow.set_tag(key,value)
            mlflow.set_tag("job", "xgboost using optuna to search parameter")
    


  0%|          | 0/100 [00:00<?, ?it/s]

2024/12/06 03:07:30 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "d:\project\mlsec\final\project\lib\site-packages\xgboost\core.py:158: UserWarning: [03:07:30] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."


🏃 View run big-hog-128 at: http://localhost:5000/#/experiments/9/runs/c4acb282001640c0a7be077ed1bd485e
🧪 View experiment at: http://localhost:5000/#/experiments/9
Initial trial 0 achieved value: 0.9333673003237396


2024/12/06 03:11:28 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "d:\project\mlsec\final\project\lib\site-packages\xgboost\core.py:158: UserWarning: [03:11:28] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."
d:\project\mlsec\final\project\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


🏃 View run judicious-shad-647 at: http://localhost:5000/#/experiments/9/runs/209b489f289c4ca89e55d508568879fb
🧪 View experiment at: http://localhost:5000/#/experiments/9


...

2024/12/06 05:06:22 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "d:\project\mlsec\final\project\lib\site-packages\xgboost\core.py:158: UserWarning: [05:06:22] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."


🏃 View run auspicious-lynx-277 at: http://localhost:5000/#/experiments/9/runs/0b3de205e9304ba3b105e1542ef0dc97
🧪 View experiment at: http://localhost:5000/#/experiments/9


C:\Users\SonNH\AppData\Local\Temp\ipykernel_7932\2088137831.py:94: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_ = client.get_latest_versions(model_name, stages=None)[0]
Registered model 'XGBoost-Classifier' already exists. Creating a new version of this model...
2024/12/06 05:06:37 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGBoost-Classifier, version 5


Last model is better. Current values 0.9401911227995349, latest values 0.9405534481927864
🏃 View run fortunate-trout-941 at: http://localhost:5000/#/experiments/9/runs/b736c3dcf1e54cff9de92fd33f746bb2
🧪 View experiment at: http://localhost:5000/#/experiments/9


Created version '5' of model 'XGBoost-Classifier'.


In [ ]:
model = XGBClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
f1 = f1_score(y_test, y_pred, average='weighted')
feature_important = model.get_booster().get_score(importance_type='weight')
keys = list(feature_important.keys())
values = list(feature_important.values())
data = pd.DataFrame(data=values, index=keys, columns=['score']).sort_values(by = 'score', ascending=False)
data.nlargest(40, columns="score").plot(kind='barh', figsize = (20,20))